In [1]:
import pickle
def calculate_wer(reference, hypothesis):
	ref_words = reference.split()
	hyp_words = hypothesis.split()
	# Counting the number of substitutions, deletions, and insertions
	substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
	deletions = len(ref_words) - len(hyp_words)
	insertions = len(hyp_words) - len(ref_words)
	# Total number of words in the reference text
	total_words = len(ref_words)
	# Calculating the Word Error Rate (WER)
	wer = (substitutions + deletions + insertions) / total_words
	return wer

def calculate_wer_from_files(reference_file, hypothesis_file):
    with open(reference_file, 'rb') as ref_file:
        references = pickle.load(ref_file)
    with open(hypothesis_file, 'rb') as hyp_file:
        hypothesiss = pickle.load(hyp_file)
    if len(hypothesiss) != 100 :
        hypothesiss = hypothesiss[0]
    
    error = [calculate_wer(reference, hypothesis) for reference, hypothesis in zip(references, hypothesiss)]
    return error

In [2]:
import glob 
import pickle
import pandas as pd
water_marked_files = sorted(glob.glob('Dataset/Watermarked/en/*/*'))
attacked_files = sorted(glob.glob('Dataset/Attacked/*/*/*'))

In [3]:
d= {}
for wfile in water_marked_files:
    for afile in attacked_files:
        key = afile.split('/')[2]+' '+ afile.split('/')[-2]  + ' '
        if 'rephrased'  in afile or 'dipper' in afile:
           key+= "".join(afile.split('/')[-1].split('_')[-1]) 
        key = key.lower().replace('.pkl' , '').replace('dipper' , 'rephrased').replace('sirrephrased' , 'rephrased').title()
        key = tuple(key.split(' '))
        if ('sir' in wfile.lower() and 'sir' in afile.lower()):
             d[key] = calculate_wer_from_files(wfile, afile)
        elif('semantics' not in wfile.lower() and 'semantics' not in afile.lower()):
               d[key] = calculate_wer_from_files(wfile, afile)
        else:
                d[key] = calculate_wer_from_files(wfile, afile)
      

In [4]:
df = pd.DataFrame(d).describe()
df = df.sort_index(axis = 1)

In [5]:
styled_df = df.style.set_table_styles([
    {'selector': 'th.level0',
     'props': [('text-align', 'center')]
    },
    {'selector': 'th',
     'props': [('background-color', '#f4f4f9'), ('color', 'black')]
    },
    {'selector': 'td',
     'props': [('background-color', '#f9f9f9')]
    },
    # Add horizontal lines after each row
    {'selector': 'tr',
     'props': [('border-bottom', '1px solid black')]
    },
     {'selector': 'td',
     'props': [('border-bottom', '1px solid black')]
    }
], overwrite=True)

In [6]:
styled_df

In [7]:
def edit_distance(str1, str2):
    len1 = len(str1)
    len2 = len(str2)
    print(len1, len2)
    
    # Create a table to store results of subproblems
    dp = [[0] * (len2 + 1) for _ in range(len1 + 1)]
    
    # Fill dp[][] in bottom up manner
    for i in range(len1 + 1):
        for j in range(len2 + 1):
            # If first string is empty, only option is to
            # insert all characters of second string
            if i == 0:
                dp[i][j] = j
                
            # If second string is empty, only option is to
            # remove all characters of first string
            elif j == 0:
                dp[i][j] = i
                
            # If last characters are same, ignore last char
            # and recur for remaining string
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
                
            # If last character are different, consider all
            # possibilities and find minimum
            else:
                dp[i][j] = 1 + min(dp[i][j - 1],      # Insert
                                   dp[i - 1][j],      # Remove
                                   dp[i - 1][j - 1])  # Replace
    
    return dp[len1][len2] / len1


def calculate_edit_from_files(reference_file, hypothesis_file):
    with open(reference_file, 'rb') as ref_file:
        references = pickle.load(ref_file)
    with open(hypothesis_file, 'rb') as hyp_file:
        hypothesiss = pickle.load(hyp_file)
    if len(hypothesiss) != 100 :
        hypothesiss = hypothesiss[0]
    
    error = [edit_distance(reference, hypothesis) for reference, hypothesis in zip(references[:1], hypothesiss[:1])]
    return error


In [8]:
d= {}
for wfile in water_marked_files:
    for afile in attacked_files:
        key = afile.split('/')[2]+' '+ afile.split('/')[-2]  + ' '
        if 'rephrased'  in afile or 'dipper' in afile:
           key+= "".join(afile.split('/')[-1].split('_')[-1]) 
        key = key.lower().replace('.pkl' , '').replace('dipper' , 'rephrased').replace('sirrephrased' , 'rephrased').title()
        key = tuple(key.split(' '))
        if ('sir' in wfile.lower() and 'sir' in afile.lower()):
             d[key] = calculate_edit_from_files(wfile, afile)
        elif('semantics' not in wfile.lower() and 'semantics' not in afile.lower()):
               d[key] = calculate_edit_from_files(wfile, afile)
        else:
                d[key] = calculate_edit_from_files(wfile, afile)
      

1481 1439
1481 1459
1481 610
1481 1040
1481 1055
1481 906
1481 1394
1481 1433
1481 574
1481 1090
1481 1083
1481 916
1481 465
1481 411
1481 398
1481 399
1481 393
1481 1359
1481 1223
1481 1184
1481 1101
1481 1057
1481 1265
1481 894
1481 883
1481 811
1481 654
1481 480
1481 398
1481 392
1481 398
1481 384
1481 1345
1481 1171
1481 1192
1481 1119
1481 1080
1481 1275
1481 870
1481 822
1481 774
1481 640
1520 1439
1520 1459
1520 610
1520 1040
1520 1055
1520 906
1520 1394
1520 1433
1520 574
1520 1090
1520 1083
1520 916
1520 465
1520 411
1520 398
1520 399
1520 393
1520 1359
1520 1223
1520 1184
1520 1101
1520 1057
1520 1265
1520 894
1520 883
1520 811
1520 654
1520 480
1520 398
1520 392
1520 398
1520 384
1520 1345
1520 1171
1520 1192
1520 1119
1520 1080
1520 1275
1520 870
1520 822
1520 774
1520 640
2190 1439
2190 1459
2190 610
2190 1040
2190 1055
2190 906
2190 1394
2190 1433
2190 574
2190 1090
2190 1083
2190 916
2190 465
2190 411
2190 398
2190 399
2190 393
2190 1359
2190 1223
2190 1184
2190 1101
219

In [9]:
d

{('Normaltranslation', 'Kwg', ''): [0.5958904109589042],
 ('Normaltranslation', 'Semantics', ''): [0.5972602739726027],
 ('Normaltranslation', 'Sir', ''): [0.8022831050228311],
 ('Paraphraed_Pivottranslation', 'Kwg', ''): [0.6579908675799087],
 ('Paraphraed_Pivottranslation', 'Semantics', ''): [0.6570776255707762],
 ('Paraphraed_Pivottranslation', 'Sir', ''): [0.6831050228310502],
 ('Paraphrased_Normaltranslation', 'Kwg', ''): [0.6273972602739726],
 ('Paraphrased_Normaltranslation', 'Semantics', ''): [0.6132420091324201],
 ('Paraphrased_Normaltranslation', 'Sir', ''): [0.828310502283105],
 ('Pivottranslation', 'Kwg', ''): [0.6515981735159817],
 ('Pivottranslation', 'Semantics', ''): [0.6534246575342466],
 ('Pivottranslation', 'Sir', ''): [0.6831050228310502],
 ('Recursivepara', 'Sir', 'Rephrased0'): [0.8529680365296803],
 ('Recursivepara', 'Sir', 'Rephrased1'): [0.8721461187214612],
 ('Recursivepara', 'Sir', 'Rephrased2'): [0.8753424657534247],
 ('Recursivepara', 'Sir', 'Rephrased3'): 

In [7]:
from bert_score import score

def calculate_bert_score_from_files(reference_file, hypothesis_file):
    with open(reference_file, 'rb') as ref_file:
        references = pickle.load(ref_file)
    with open(hypothesis_file, 'rb') as hyp_file:
        hypothesiss = pickle.load(hyp_file)
    if len(hypothesiss) != 100 :
        hypothesiss = hypothesiss[0]
    error = []
    s = score(references, hypothesiss, lang="en", verbose=True)
    error.append((s[0].item(), s[1].item(), s[2].item()))
    return error

d= {}
for wfile in water_marked_files:
    for afile in attacked_files:
        key = afile.split('/')[2]+' '+ afile.split('/')[-2]  + ' '
        if 'rephrased'  in afile or 'dipper' in afile:
           key+= "".join(afile.split('/')[-1].split('_')[-1]) 
        key = key.lower().replace('.pkl' , '').replace('dipper' , 'rephrased').replace('sirrephrased' , 'rephrased').title()
        key = tuple(key.split(' '))
        if ('sir' in wfile.lower() and 'sir' in afile.lower()):
             d[key] = calculate_bert_score_from_files(wfile, afile)
        elif('semantics' not in wfile.lower() and 'semantics' not in afile.lower()):
               d[key] = calculate_bert_score_from_files(wfile, afile)
        else:
                d[key] = calculate_bert_score_from_files(wfile, afile)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
d

{('Normaltranslation',
  'Kwg',
  ''): [(tensor([0.8491]), tensor([0.8962]), tensor([0.8720]))],
 ('Normaltranslation',
  'Semantics',
  ''): [(tensor([0.8431]), tensor([0.8918]), tensor([0.8668]))],
 ('Normaltranslation',
  'Sir',
  ''): [(tensor([0.8158]), tensor([0.8164]), tensor([0.8161]))],
 ('Paraphraed_Pivottranslation',
  'Kwg',
  ''): [(tensor([0.8379]), tensor([0.9094]), tensor([0.8722]))],
 ('Paraphraed_Pivottranslation',
  'Semantics',
  ''): [(tensor([0.8340]), tensor([0.8927]), tensor([0.8624]))],
 ('Paraphraed_Pivottranslation',
  'Sir',
  ''): [(tensor([0.8339]), tensor([0.9054]), tensor([0.8682]))],
 ('Paraphrased_Normaltranslation',
  'Kwg',
  ''): [(tensor([0.8369]), tensor([0.8800]), tensor([0.8579]))],
 ('Paraphrased_Normaltranslation',
  'Semantics',
  ''): [(tensor([0.8327]), tensor([0.8760]), tensor([0.8538]))],
 ('Paraphrased_Normaltranslation',
  'Sir',
  ''): [(tensor([0.8085]), tensor([0.7948]), tensor([0.8016]))],
 ('Pivottranslation',
  'Kwg',
  ''): [(ten